In [1]:
import os, gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
from pathlib import Path

In [2]:
%pwd

'c:\\Users\\takashi\\Documents\\Kaggle\\hms-harmful-brain-activity-classification\\working'

In [3]:
!chcp 65001
!dir
%ls

Active code page: 65001
 Volume in drive C is Windows
 Volume Serial Number is 3E25-FCC2

 Directory of c:\Users\takashi\Documents\Kaggle\hms-harmful-brain-activity-classification\working

2024/03/04  14:50    <DIR>          .
2024/02/26  11:29    <DIR>          ..
2024/03/04  14:50           163,743 catboost-eeg-feature-ver1-inference.ipynb
2024/03/04  14:50           147,459 catboost-eeg-feature-ver1-train.ipynb
2024/02/26  11:28           135,240 catboost-starter-lb-0-67.ipynb
2024/02/26  11:28            33,637 catboost-starter-ver3-inference.ipynb
2024/02/26  11:28           125,279 catboost-starter-ver3-train.ipynb
2024/02/29  01:13    <DIR>          catboost_info
2024/02/28  23:11           104,962 eda-catboost-feature-all-diffs-in-each-channel.ipynb
2024/03/05  00:44           303,609 eda-catboost-feature-base.ipynb
2024/02/26  11:28         2,551,404 wavenet-starter-lb-0-66.ipynb
2024/03/04  14:50           163,743 wrong_catboost-eeg-feature-ver1-inference.ipynb
2024/03/04  14

In [4]:
temp = pl.read_parquet("../input/hms-harmful-brain-activity-classification/train_eegs/1001717358.parquet")
temp.head()

Fp1,F3,C3,P3,F7,T3,T5,O1,Fz,Cz,Pz,Fp2,F4,C4,P4,F8,T4,T6,O2,EKG
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,0.0
9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,0.0
9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,0.0
9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,0.0
9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,0.0


In [5]:
df_train = pl.read_csv("../input/hms-harmful-brain-activity-classification/train.csv")
df_train.head()

eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
i64,i64,f64,i64,i64,f64,i64,i64,str,i64,i64,i64,i64,i64,i64
1628180742,0,0.0,353733,0,0.0,127492639,42516,"""Seizure""",3,0,0,0,0,0
1628180742,1,6.0,353733,1,6.0,3887563113,42516,"""Seizure""",3,0,0,0,0,0
1628180742,2,8.0,353733,2,8.0,1142670488,42516,"""Seizure""",3,0,0,0,0,0
1628180742,3,18.0,353733,3,18.0,2718991173,42516,"""Seizure""",3,0,0,0,0,0
1628180742,4,24.0,353733,4,24.0,3080632009,42516,"""Seizure""",3,0,0,0,0,0


In [6]:
df_train = df_train.group_by(by = "eeg_id").agg(pl.first("expert_consensus"))
df_train.head()

eeg_id,expert_consensus
i64,str
3161926366,"""Other"""
3542452428,"""LPD"""
3046277567,"""Other"""
2934282497,"""GRDA"""
1103440233,"""Other"""


In [7]:
PATH = '../input/hms-harmful-brain-activity-classification/train_eegs/'
files = os.listdir(PATH)
print(f'There are {len(files)} eeg parquet files')

all_eegs = {}
temp = pl.DataFrame()
for i, f in enumerate(files):
    if i%100 == 0: print(i, ', ', end = '')

    eeg_id = int(f.split('.')[0])    
    if i%20 == 7:
    # if (i < 1) or (eeg_id == 568657):
    # if eeg_id == 568657:
        temp_eeg = pl.read_parquet(f'{PATH}{f}')
    
        # eegデータにおける、計測開始から各行までの秒数 seconds_eeg
        t = 1/200 # EEGのサンプリング周波数は200Hzなので、1行は1/200秒ごとの計測値を表す  
        seconds = [i*t for i in range(len(temp_eeg))]
        max_second = max(seconds)
        temp_eeg = temp_eeg.with_columns(pl.Series(seconds).alias("seconds_eeg")) 
        

        # 差分特徴量を作る
        temp_eeg = temp_eeg.with_columns(
            (pl.col("Fp1") - pl.col("T3")).alias("LL_Fp1-T3"),
            (pl.col("T3") - pl.col("O1")).alias("LL_T3-O1"),
            (pl.col("Fp1") - pl.col("C3")).alias("LP_Fp1-C3"),
            (pl.col("C3") - pl.col("O1")).alias("LP_C3-O1"),
            (pl.col("Fp2") - pl.col("C4")).alias("RP_Fp2-C4"),
            (pl.col("C4") - pl.col("O2")).alias("RP_C4-O2"),
            (pl.col("Fp2") - pl.col("T4")).alias("RR_Fp2-T4"),
            (pl.col("T4") - pl.col("O2")).alias("RR_T4-O2"),
        )

        # 差分特徴量の集計特徴量をつくる
        # window = 4000 # 20秒窓 (20秒 / (1レコード1/200秒) = 4000レコード)
        window_list = [2000, 4000]

        for window in window_list:

            str_window = str(int(window / (200))) + "s"
            temp_eeg = temp_eeg.with_columns(
                # LL: Fp1 - T3
                pl.col("LL_Fp1-T3").rolling_mean(window_size=window, center=True).alias("mean_" + str_window + "_LL_Fp1-T3"),
                pl.col("LL_Fp1-T3").rolling_max(window_size=window, center=True).alias("min_" + str_window + "_LL_Fp1-T3"),
                pl.col("LL_Fp1-T3").rolling_min(window_size=window, center=True).alias("max_" + str_window + "_LL_Fp1-T3"),
                pl.col("LL_Fp1-T3").rolling_std(window_size=window, center=True).alias("std_" + str_window + "_LL_Fp1-T3"),
                (pl.col("LL_Fp1-T3").rolling_max(window_size=window, center=True) - pl.col("LL_Fp1-T3").rolling_min(window_size=window, center=True)).alias("max-min_" + str_window + "_LL_Fp1-T3"),

                # LL: Fp1 - T3
                pl.col("LL_T3-O1").rolling_mean(window_size=window, center=True).alias("mean_" + str_window + "_LL_T3-O1"),
                pl.col("LL_T3-O1").rolling_max(window_size=window, center=True).alias("min_" + str_window + "_LL_T3-O1"),
                pl.col("LL_T3-O1").rolling_min(window_size=window, center=True).alias("max_" + str_window + "_LL_T3-O1"),
                pl.col("LL_T3-O1").rolling_std(window_size=window, center=True).alias("std_" + str_window + "_LL_T3-O1"),
                (pl.col("LL_T3-O1").rolling_max(window_size=window, center=True) - pl.col("LL_T3-O1").rolling_min(window_size=window, center=True)).alias("max-min__" + str_window + "_LL_T3-O1"),


                # LP: Fp1 - C3
                pl.col("LP_Fp1-C3").rolling_mean(window_size=window, center=True).alias("mean_" + str_window + "_LP_Fp1-C3"),
                pl.col("LP_Fp1-C3").rolling_max(window_size=window, center=True).alias("min_" + str_window + "_LP_Fp1-C3"),
                pl.col("LP_Fp1-C3").rolling_min(window_size=window, center=True).alias("max_" + str_window + "_LP_Fp1-C3"),
                pl.col("LP_Fp1-C3").rolling_std(window_size=window, center=True).alias("std_" + str_window + "_LP_Fp1-C3"),
                (pl.col("LP_Fp1-C3").rolling_max(window_size=window, center=True) - pl.col("LP_Fp1-C3").rolling_min(window_size=window, center=True)).alias("max-min_" + str_window + "_LP_Fp1-C3"),

                # LP: C3 - O1
                pl.col("LP_C3-O1").rolling_mean(window_size=window, center=True).alias("mean_" + str_window + "_LP_C3-O1"),
                pl.col("LP_C3-O1").rolling_max(window_size=window, center=True).alias("min_" + str_window + "_LP_C3-O1"),
                pl.col("LP_C3-O1").rolling_min(window_size=window, center=True).alias("max_" + str_window + "_LP_C3-O1"),
                pl.col("LP_C3-O1").rolling_std(window_size=window, center=True).alias("std_" + str_window + "_LP_C3-O1"),
                (pl.col("LP_C3-O1").rolling_max(window_size=window, center=True) - pl.col("LP_C3-O1").rolling_min(window_size=window, center=True)).alias("max-min_" + str_window + "_LP_C3-O1"),


                # RP: Fp1 - C4
                pl.col("RP_Fp2-C4").rolling_mean(window_size=window, center=True).alias("mean_" + str_window + "_RP_Fp2-C4"),
                pl.col("RP_Fp2-C4").rolling_max(window_size=window, center=True).alias("min_" + str_window + "_RP_Fp2-C4"),
                pl.col("RP_Fp2-C4").rolling_min(window_size=window, center=True).alias("max_" + str_window + "_RP_Fp2-C4"),
                pl.col("RP_Fp2-C4").rolling_std(window_size=window, center=True).alias("std_" + str_window + "_RP_Fp2-C4"),
                (pl.col("RP_Fp2-C4").rolling_max(window_size=window, center=True) - pl.col("RP_Fp2-C4").rolling_min(window_size=window, center=True)).alias("max-min_" + str_window + "_RP_Fp2-C4"),

                # RP: C4 - O2
                pl.col("RP_C4-O2").rolling_mean(window_size=window, center=True).alias("mean_" + str_window + "_RP_C4-O2"),
                pl.col("RP_C4-O2").rolling_max(window_size=window, center=True).alias("min_" + str_window + "_RP_C4-O2"),
                pl.col("RP_C4-O2").rolling_min(window_size=window, center=True).alias("max_" + str_window + "_RP_C4-O2"),
                pl.col("RP_C4-O2").rolling_std(window_size=window, center=True).alias("std_" + str_window + "_RP_C4-O2"),
                (pl.col("RP_C4-O2").rolling_max(window_size=window, center=True) - pl.col("RP_C4-O2").rolling_min(window_size=window, center=True)).alias("max-min_" + str_window + "_RP_C4-O2"),


                # RR: Fp2 - T4
                pl.col("RR_Fp2-T4").rolling_mean(window_size=window, center=True).alias("mean_" + str_window + "_RR_Fp2-T4"),
                pl.col("RR_Fp2-T4").rolling_max(window_size=window, center=True).alias("min_" + str_window + "_RR_Fp2-T4"),
                pl.col("RR_Fp2-T4").rolling_min(window_size=window, center=True).alias("max_" + str_window + "_RR_Fp2-T4"),
                pl.col("RR_Fp2-T4").rolling_std(window_size=window, center=True).alias("std_" + str_window + "_RR_Fp2-T4"),
                (pl.col("RR_Fp2-T4").rolling_max(window_size=window, center=True) - pl.col("RR_Fp2-T4").rolling_min(window_size=window, center=True)).alias("max-min_" + str_window + "_RR_Fp2-T4"),

                # RR: T4 - O2
                pl.col("RR_T4-O2").rolling_mean(window_size=window, center=True).alias("mean_" + str_window + "_RR_T4-O2"),
                pl.col("RR_T4-O2").rolling_max(window_size=window, center=True).alias("min_" + str_window + "_RR_T4-O2"),
                pl.col("RR_T4-O2").rolling_min(window_size=window, center=True).alias("max_" + str_window + "_RR_T4-O2"),
                pl.col("RR_T4-O2").rolling_std(window_size=window, center=True).alias("std_" + str_window + "_RR_T4-O2"),
                (pl.col("RR_T4-O2").rolling_max(window_size=window, center=True) - pl.col("RR_T4-O2").rolling_min(window_size=window, center=True)).alias("max-min_" + str_window + "_RR_T4-O2"),
            )

        # 集計特徴量とEKG以外drop
        temp_eeg = temp_eeg.drop(
            ['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2',
             'LL_Fp1-T3', 'LL_T3-O1', 'LP_Fp1-C3', 'LP_C3-O1', 'RP_Fp2-C4', 'RP_C4-O2', 'RR_Fp2-T4', 'RR_T4-O2'])
        
        # eeg_id を追加
            # 型        
        temp_eeg = temp_eeg.with_columns(pl.lit(str(eeg_id)).alias("eeg_id"))

        # 全ての
        temp = pl.concat([temp, temp_eeg], how = 'vertical')

    else:
        continue

There are 17300 eeg parquet files
0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , 3900 , 4000 , 4100 , 4200 , 4300 , 4400 , 4500 , 4600 , 4700 , 4800 , 4900 , 5000 , 5100 , 5200 , 5300 , 5400 , 5500 , 5600 , 5700 , 5800 , 5900 , 6000 , 6100 , 6200 , 6300 , 6400 , 6500 , 6600 , 6700 , 6800 , 6900 , 7000 , 7100 , 7200 , 7300 , 7400 , 7500 , 7600 , 7700 , 7800 , 7900 , 8000 , 8100 , 8200 , 8300 , 8400 , 8500 , 8600 , 8700 , 8800 , 8900 , 9000 , 9100 , 9200 , 9300 , 9400 , 9500 , 9600 , 9700 , 9800 , 9900 , 10000 , 10100 , 10200 , 10300 , 10400 , 10500 , 10600 , 10700 , 10800 , 10900 , 11000 , 11100 , 11200 , 11300 , 11400 , 11500 , 11600 , 11700 , 11800 , 11900 , 12000 , 12100 , 12200 , 12300 , 12400 , 12500 , 12600 , 12700 , 12800 , 12900 , 13000 , 13100 , 13200 , 13300 , 13400 

In [8]:
temp = temp.with_columns(pl.col("eeg_id").cast(pl.Int64).alias("eeg_id"))
temp.tail(5)

EKG,seconds_eeg,mean_10s_LL_Fp1-T3,min_10s_LL_Fp1-T3,max_10s_LL_Fp1-T3,std_10s_LL_Fp1-T3,max-min_10s_LL_Fp1-T3,mean_10s_LL_T3-O1,min_10s_LL_T3-O1,max_10s_LL_T3-O1,std_10s_LL_T3-O1,max-min__10s_LL_T3-O1,mean_10s_LP_Fp1-C3,min_10s_LP_Fp1-C3,max_10s_LP_Fp1-C3,std_10s_LP_Fp1-C3,max-min_10s_LP_Fp1-C3,mean_10s_LP_C3-O1,min_10s_LP_C3-O1,max_10s_LP_C3-O1,std_10s_LP_C3-O1,max-min_10s_LP_C3-O1,mean_10s_RP_Fp2-C4,min_10s_RP_Fp2-C4,max_10s_RP_Fp2-C4,std_10s_RP_Fp2-C4,max-min_10s_RP_Fp2-C4,mean_10s_RP_C4-O2,min_10s_RP_C4-O2,max_10s_RP_C4-O2,std_10s_RP_C4-O2,max-min_10s_RP_C4-O2,mean_10s_RR_Fp2-T4,min_10s_RR_Fp2-T4,max_10s_RR_Fp2-T4,std_10s_RR_Fp2-T4,max-min_10s_RR_Fp2-T4,…,max-min_20s_LL_Fp1-T3,mean_20s_LL_T3-O1,min_20s_LL_T3-O1,max_20s_LL_T3-O1,std_20s_LL_T3-O1,max-min__20s_LL_T3-O1,mean_20s_LP_Fp1-C3,min_20s_LP_Fp1-C3,max_20s_LP_Fp1-C3,std_20s_LP_Fp1-C3,max-min_20s_LP_Fp1-C3,mean_20s_LP_C3-O1,min_20s_LP_C3-O1,max_20s_LP_C3-O1,std_20s_LP_C3-O1,max-min_20s_LP_C3-O1,mean_20s_RP_Fp2-C4,min_20s_RP_Fp2-C4,max_20s_RP_Fp2-C4,std_20s_RP_Fp2-C4,max-min_20s_RP_Fp2-C4,mean_20s_RP_C4-O2,min_20s_RP_C4-O2,max_20s_RP_C4-O2,std_20s_RP_C4-O2,max-min_20s_RP_C4-O2,mean_20s_RR_Fp2-T4,min_20s_RR_Fp2-T4,max_20s_RR_Fp2-T4,std_20s_RR_Fp2-T4,max-min_20s_RR_Fp2-T4,mean_20s_RR_T4-O2,min_20s_RR_T4-O2,max_20s_RR_T4-O2,std_20s_RR_T4-O2,max-min_20s_RR_T4-O2,eeg_id
f32,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64
146.5,121.975,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,9980241
52.720001,121.98,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,9980241
46.68,121.985,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,9980241
142.729996,121.99,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,9980241
90.68,121.995,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,9980241


In [9]:
temp.filter(pl.col("eeg_id") == 568657).filter(pl.col("seconds_eeg") == 10)

EKG,seconds_eeg,mean_10s_LL_Fp1-T3,min_10s_LL_Fp1-T3,max_10s_LL_Fp1-T3,std_10s_LL_Fp1-T3,max-min_10s_LL_Fp1-T3,mean_10s_LL_T3-O1,min_10s_LL_T3-O1,max_10s_LL_T3-O1,std_10s_LL_T3-O1,max-min__10s_LL_T3-O1,mean_10s_LP_Fp1-C3,min_10s_LP_Fp1-C3,max_10s_LP_Fp1-C3,std_10s_LP_Fp1-C3,max-min_10s_LP_Fp1-C3,mean_10s_LP_C3-O1,min_10s_LP_C3-O1,max_10s_LP_C3-O1,std_10s_LP_C3-O1,max-min_10s_LP_C3-O1,mean_10s_RP_Fp2-C4,min_10s_RP_Fp2-C4,max_10s_RP_Fp2-C4,std_10s_RP_Fp2-C4,max-min_10s_RP_Fp2-C4,mean_10s_RP_C4-O2,min_10s_RP_C4-O2,max_10s_RP_C4-O2,std_10s_RP_C4-O2,max-min_10s_RP_C4-O2,mean_10s_RR_Fp2-T4,min_10s_RR_Fp2-T4,max_10s_RR_Fp2-T4,std_10s_RR_Fp2-T4,max-min_10s_RR_Fp2-T4,…,max-min_20s_LL_Fp1-T3,mean_20s_LL_T3-O1,min_20s_LL_T3-O1,max_20s_LL_T3-O1,std_20s_LL_T3-O1,max-min__20s_LL_T3-O1,mean_20s_LP_Fp1-C3,min_20s_LP_Fp1-C3,max_20s_LP_Fp1-C3,std_20s_LP_Fp1-C3,max-min_20s_LP_Fp1-C3,mean_20s_LP_C3-O1,min_20s_LP_C3-O1,max_20s_LP_C3-O1,std_20s_LP_C3-O1,max-min_20s_LP_C3-O1,mean_20s_RP_Fp2-C4,min_20s_RP_Fp2-C4,max_20s_RP_Fp2-C4,std_20s_RP_Fp2-C4,max-min_20s_RP_Fp2-C4,mean_20s_RP_C4-O2,min_20s_RP_C4-O2,max_20s_RP_C4-O2,std_20s_RP_C4-O2,max-min_20s_RP_C4-O2,mean_20s_RR_Fp2-T4,min_20s_RR_Fp2-T4,max_20s_RR_Fp2-T4,std_20s_RR_Fp2-T4,max-min_20s_RR_Fp2-T4,mean_20s_RR_T4-O2,min_20s_RR_T4-O2,max_20s_RR_T4-O2,std_20s_RR_T4-O2,max-min_20s_RR_T4-O2,eeg_id
f32,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64


In [10]:
center_seconds_list = [20, 30, 40]
for sec in center_seconds_list:
    # ある時刻における集計特徴量を取得
    temp_sec = temp.filter(pl.col("seconds_eeg") == sec)
    temp_sec = temp_sec.drop(["EKG", "seconds_eeg"])
    
    print(temp_sec.head())

    # 列名を一括で変更(eeg_idはもとに戻す)
    col_suffix = "_at_" + str(sec) + "s"
    col_eeg_id = "eeg_id_at_" + str(sec) + "s"
    list_original_col = temp_sec.columns
    temp_sec = temp_sec.with_columns(pl.all().name.suffix(col_suffix)).drop(list_original_col) # suffixを付与して、元の列名の列を削除
    temp_sec = temp_sec.rename({col_eeg_id: "eeg_id"})

    print(df_train.shape)

    # join
    df_train = df_train.join(temp_sec, on = "eeg_id", how = "left")

    print(df_train.shape)

shape: (5, 81)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ mean_10s_ ┆ min_10s_L ┆ max_10s_L ┆ std_10s_L ┆ … ┆ max_20s_R ┆ std_20s_R ┆ max-min_2 ┆ eeg_id   │
│ LL_Fp1-T3 ┆ L_Fp1-T3  ┆ L_Fp1-T3  ┆ L_Fp1-T3  ┆   ┆ R_T4-O2   ┆ R_T4-O2   ┆ 0s_RR_T4- ┆ ---      │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ O2        ┆ i64      │
│ f32       ┆ f32       ┆ f32       ┆ f32       ┆   ┆ f32       ┆ f32       ┆ ---       ┆          │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆ f32       ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ -0.728921 ┆ 110.12000 ┆ -120.7900 ┆ 33.757694 ┆ … ┆ -97.10000 ┆ 25.106779 ┆ 205.18000 ┆ 10023790 │
│           ┆ 3         ┆ 01        ┆           ┆   ┆ 6         ┆           ┆ 8         ┆ 34       │
│ 0.25241   ┆ 132.80999 ┆ -150.1999 ┆ 65.981064 ┆ … ┆ -74.84999 ┆ 15.500906 

In [11]:
pl.Config.set_tbl_cols(-1)
df_train.head()

eeg_id,expert_consensus,mean_10s_LL_Fp1-T3_at_20s,min_10s_LL_Fp1-T3_at_20s,max_10s_LL_Fp1-T3_at_20s,std_10s_LL_Fp1-T3_at_20s,max-min_10s_LL_Fp1-T3_at_20s,mean_10s_LL_T3-O1_at_20s,min_10s_LL_T3-O1_at_20s,max_10s_LL_T3-O1_at_20s,std_10s_LL_T3-O1_at_20s,max-min__10s_LL_T3-O1_at_20s,mean_10s_LP_Fp1-C3_at_20s,min_10s_LP_Fp1-C3_at_20s,max_10s_LP_Fp1-C3_at_20s,std_10s_LP_Fp1-C3_at_20s,max-min_10s_LP_Fp1-C3_at_20s,mean_10s_LP_C3-O1_at_20s,min_10s_LP_C3-O1_at_20s,max_10s_LP_C3-O1_at_20s,std_10s_LP_C3-O1_at_20s,max-min_10s_LP_C3-O1_at_20s,mean_10s_RP_Fp2-C4_at_20s,min_10s_RP_Fp2-C4_at_20s,max_10s_RP_Fp2-C4_at_20s,std_10s_RP_Fp2-C4_at_20s,max-min_10s_RP_Fp2-C4_at_20s,mean_10s_RP_C4-O2_at_20s,min_10s_RP_C4-O2_at_20s,max_10s_RP_C4-O2_at_20s,std_10s_RP_C4-O2_at_20s,max-min_10s_RP_C4-O2_at_20s,mean_10s_RR_Fp2-T4_at_20s,min_10s_RR_Fp2-T4_at_20s,max_10s_RR_Fp2-T4_at_20s,std_10s_RR_Fp2-T4_at_20s,max-min_10s_RR_Fp2-T4_at_20s,mean_10s_RR_T4-O2_at_20s,min_10s_RR_T4-O2_at_20s,max_10s_RR_T4-O2_at_20s,std_10s_RR_T4-O2_at_20s,max-min_10s_RR_T4-O2_at_20s,mean_20s_LL_Fp1-T3_at_20s,min_20s_LL_Fp1-T3_at_20s,max_20s_LL_Fp1-T3_at_20s,std_20s_LL_Fp1-T3_at_20s,max-min_20s_LL_Fp1-T3_at_20s,mean_20s_LL_T3-O1_at_20s,min_20s_LL_T3-O1_at_20s,max_20s_LL_T3-O1_at_20s,std_20s_LL_T3-O1_at_20s,max-min__20s_LL_T3-O1_at_20s,mean_20s_LP_Fp1-C3_at_20s,min_20s_LP_Fp1-C3_at_20s,max_20s_LP_Fp1-C3_at_20s,std_20s_LP_Fp1-C3_at_20s,max-min_20s_LP_Fp1-C3_at_20s,mean_20s_LP_C3-O1_at_20s,min_20s_LP_C3-O1_at_20s,max_20s_LP_C3-O1_at_20s,std_20s_LP_C3-O1_at_20s,max-min_20s_LP_C3-O1_at_20s,mean_20s_RP_Fp2-C4_at_20s,min_20s_RP_Fp2-C4_at_20s,max_20s_RP_Fp2-C4_at_20s,std_20s_RP_Fp2-C4_at_20s,max-min_20s_RP_Fp2-C4_at_20s,mean_20s_RP_C4-O2_at_20s,min_20s_RP_C4-O2_at_20s,max_20s_RP_C4-O2_at_20s,std_20s_RP_C4-O2_at_20s,max-min_20s_RP_C4-O2_at_20s,mean_20s_RR_Fp2-T4_at_20s,min_20s_RR_Fp2-T4_at_20s,max_20s_RR_Fp2-T4_at_20s,std_20s_RR_Fp2-T4_at_20s,max-min_20s_RR_Fp2-T4_at_20s,mean_20s_RR_T4-O2_at_20s,min_20s_RR_T4-O2_at_20s,max_20s_RR_T4-O2_at_20s,std_20s_RR_T4-O2_at_20s,max-min_20s_RR_T4-O2_at_20s,mean_10s_LL_Fp1-T3_at_30s,min_10s_LL_Fp1-T3_at_30s,max_10s_LL_Fp1-T3_at_30s,std_10s_LL_Fp1-T3_at_30s,max-min_10s_LL_Fp1-T3_at_30s,mean_10s_LL_T3-O1_at_30s,min_10s_LL_T3-O1_at_30s,max_10s_LL_T3-O1_at_30s,std_10s_LL_T3-O1_at_30s,max-min__10s_LL_T3-O1_at_30s,mean_10s_LP_Fp1-C3_at_30s,min_10s_LP_Fp1-C3_at_30s,max_10s_LP_Fp1-C3_at_30s,std_10s_LP_Fp1-C3_at_30s,max-min_10s_LP_Fp1-C3_at_30s,mean_10s_LP_C3-O1_at_30s,min_10s_LP_C3-O1_at_30s,max_10s_LP_C3-O1_at_30s,std_10s_LP_C3-O1_at_30s,max-min_10s_LP_C3-O1_at_30s,mean_10s_RP_Fp2-C4_at_30s,min_10s_RP_Fp2-C4_at_30s,max_10s_RP_Fp2-C4_at_30s,std_10s_RP_Fp2-C4_at_30s,max-min_10s_RP_Fp2-C4_at_30s,mean_10s_RP_C4-O2_at_30s,min_10s_RP_C4-O2_at_30s,max_10s_RP_C4-O2_at_30s,std_10s_RP_C4-O2_at_30s,max-min_10s_RP_C4-O2_at_30s,mean_10s_RR_Fp2-T4_at_30s,min_10s_RR_Fp2-T4_at_30s,max_10s_RR_Fp2-T4_at_30s,std_10s_RR_Fp2-T4_at_30s,max-min_10s_RR_Fp2-T4_at_30s,mean_10s_RR_T4-O2_at_30s,min_10s_RR_T4-O2_at_30s,max_10s_RR_T4-O2_at_30s,std_10s_RR_T4-O2_at_30s,max-min_10s_RR_T4-O2_at_30s,mean_20s_LL_Fp1-T3_at_30s,min_20s_LL_Fp1-T3_at_30s,max_20s_LL_Fp1-T3_at_30s,std_20s_LL_Fp1-T3_at_30s,max-min_20s_LL_Fp1-T3_at_30s,mean_20s_LL_T3-O1_at_30s,min_20s_LL_T3-O1_at_30s,max_20s_LL_T3-O1_at_30s,std_20s_LL_T3-O1_at_30s,max-min__20s_LL_T3-O1_at_30s,mean_20s_LP_Fp1-C3_at_30s,min_20s_LP_Fp1-C3_at_30s,max_20s_LP_Fp1-C3_at_30s,std_20s_LP_Fp1-C3_at_30s,max-min_20s_LP_Fp1-C3_at_30s,mean_20s_LP_C3-O1_at_30s,min_20s_LP_C3-O1_at_30s,max_20s_LP_C3-O1_at_30s,std_20s_LP_C3-O1_at_30s,max-min_20s_LP_C3-O1_at_30s,mean_20s_RP_Fp2-C4_at_30s,min_20s_RP_Fp2-C4_at_30s,max_20s_RP_Fp2-C4_at_30s,std_20s_RP_Fp2-C4_at_30s,max-min_20s_RP_Fp2-C4_at_30s,mean_20s_RP_C4-O2_at_30s,min_20s_RP_C4-O2_at_30s,max_20s_RP_C4-O2_at_30s,std_20s_RP_C4-O2_at_30s,max-min_20s_RP_C4-O2_at_30s,mean_20s_RR_Fp2-T4_at_30s,min_20s_RR_Fp2-T4_at_30s,max_20s_RR_Fp2-T4_at_30s,std_20s_RR_Fp2-T4_at_30s,max-min_20s_RR_Fp2-T4_at_30s,mean_20s_RR_T4-O2

In [12]:
df_train.filter(pl.col("eeg_id") == 568657)

eeg_id,expert_consensus,mean_10s_LL_Fp1-T3_at_20s,min_10s_LL_Fp1-T3_at_20s,max_10s_LL_Fp1-T3_at_20s,std_10s_LL_Fp1-T3_at_20s,max-min_10s_LL_Fp1-T3_at_20s,mean_10s_LL_T3-O1_at_20s,min_10s_LL_T3-O1_at_20s,max_10s_LL_T3-O1_at_20s,std_10s_LL_T3-O1_at_20s,max-min__10s_LL_T3-O1_at_20s,mean_10s_LP_Fp1-C3_at_20s,min_10s_LP_Fp1-C3_at_20s,max_10s_LP_Fp1-C3_at_20s,std_10s_LP_Fp1-C3_at_20s,max-min_10s_LP_Fp1-C3_at_20s,mean_10s_LP_C3-O1_at_20s,min_10s_LP_C3-O1_at_20s,max_10s_LP_C3-O1_at_20s,std_10s_LP_C3-O1_at_20s,max-min_10s_LP_C3-O1_at_20s,mean_10s_RP_Fp2-C4_at_20s,min_10s_RP_Fp2-C4_at_20s,max_10s_RP_Fp2-C4_at_20s,std_10s_RP_Fp2-C4_at_20s,max-min_10s_RP_Fp2-C4_at_20s,mean_10s_RP_C4-O2_at_20s,min_10s_RP_C4-O2_at_20s,max_10s_RP_C4-O2_at_20s,std_10s_RP_C4-O2_at_20s,max-min_10s_RP_C4-O2_at_20s,mean_10s_RR_Fp2-T4_at_20s,min_10s_RR_Fp2-T4_at_20s,max_10s_RR_Fp2-T4_at_20s,std_10s_RR_Fp2-T4_at_20s,max-min_10s_RR_Fp2-T4_at_20s,mean_10s_RR_T4-O2_at_20s,min_10s_RR_T4-O2_at_20s,max_10s_RR_T4-O2_at_20s,std_10s_RR_T4-O2_at_20s,max-min_10s_RR_T4-O2_at_20s,mean_20s_LL_Fp1-T3_at_20s,min_20s_LL_Fp1-T3_at_20s,max_20s_LL_Fp1-T3_at_20s,std_20s_LL_Fp1-T3_at_20s,max-min_20s_LL_Fp1-T3_at_20s,mean_20s_LL_T3-O1_at_20s,min_20s_LL_T3-O1_at_20s,max_20s_LL_T3-O1_at_20s,std_20s_LL_T3-O1_at_20s,max-min__20s_LL_T3-O1_at_20s,mean_20s_LP_Fp1-C3_at_20s,min_20s_LP_Fp1-C3_at_20s,max_20s_LP_Fp1-C3_at_20s,std_20s_LP_Fp1-C3_at_20s,max-min_20s_LP_Fp1-C3_at_20s,mean_20s_LP_C3-O1_at_20s,min_20s_LP_C3-O1_at_20s,max_20s_LP_C3-O1_at_20s,std_20s_LP_C3-O1_at_20s,max-min_20s_LP_C3-O1_at_20s,mean_20s_RP_Fp2-C4_at_20s,min_20s_RP_Fp2-C4_at_20s,max_20s_RP_Fp2-C4_at_20s,std_20s_RP_Fp2-C4_at_20s,max-min_20s_RP_Fp2-C4_at_20s,mean_20s_RP_C4-O2_at_20s,min_20s_RP_C4-O2_at_20s,max_20s_RP_C4-O2_at_20s,std_20s_RP_C4-O2_at_20s,max-min_20s_RP_C4-O2_at_20s,mean_20s_RR_Fp2-T4_at_20s,min_20s_RR_Fp2-T4_at_20s,max_20s_RR_Fp2-T4_at_20s,std_20s_RR_Fp2-T4_at_20s,max-min_20s_RR_Fp2-T4_at_20s,mean_20s_RR_T4-O2_at_20s,min_20s_RR_T4-O2_at_20s,max_20s_RR_T4-O2_at_20s,std_20s_RR_T4-O2_at_20s,max-min_20s_RR_T4-O2_at_20s,mean_10s_LL_Fp1-T3_at_30s,min_10s_LL_Fp1-T3_at_30s,max_10s_LL_Fp1-T3_at_30s,std_10s_LL_Fp1-T3_at_30s,max-min_10s_LL_Fp1-T3_at_30s,mean_10s_LL_T3-O1_at_30s,min_10s_LL_T3-O1_at_30s,max_10s_LL_T3-O1_at_30s,std_10s_LL_T3-O1_at_30s,max-min__10s_LL_T3-O1_at_30s,mean_10s_LP_Fp1-C3_at_30s,min_10s_LP_Fp1-C3_at_30s,max_10s_LP_Fp1-C3_at_30s,std_10s_LP_Fp1-C3_at_30s,max-min_10s_LP_Fp1-C3_at_30s,mean_10s_LP_C3-O1_at_30s,min_10s_LP_C3-O1_at_30s,max_10s_LP_C3-O1_at_30s,std_10s_LP_C3-O1_at_30s,max-min_10s_LP_C3-O1_at_30s,mean_10s_RP_Fp2-C4_at_30s,min_10s_RP_Fp2-C4_at_30s,max_10s_RP_Fp2-C4_at_30s,std_10s_RP_Fp2-C4_at_30s,max-min_10s_RP_Fp2-C4_at_30s,mean_10s_RP_C4-O2_at_30s,min_10s_RP_C4-O2_at_30s,max_10s_RP_C4-O2_at_30s,std_10s_RP_C4-O2_at_30s,max-min_10s_RP_C4-O2_at_30s,mean_10s_RR_Fp2-T4_at_30s,min_10s_RR_Fp2-T4_at_30s,max_10s_RR_Fp2-T4_at_30s,std_10s_RR_Fp2-T4_at_30s,max-min_10s_RR_Fp2-T4_at_30s,mean_10s_RR_T4-O2_at_30s,min_10s_RR_T4-O2_at_30s,max_10s_RR_T4-O2_at_30s,std_10s_RR_T4-O2_at_30s,max-min_10s_RR_T4-O2_at_30s,mean_20s_LL_Fp1-T3_at_30s,min_20s_LL_Fp1-T3_at_30s,max_20s_LL_Fp1-T3_at_30s,std_20s_LL_Fp1-T3_at_30s,max-min_20s_LL_Fp1-T3_at_30s,mean_20s_LL_T3-O1_at_30s,min_20s_LL_T3-O1_at_30s,max_20s_LL_T3-O1_at_30s,std_20s_LL_T3-O1_at_30s,max-min__20s_LL_T3-O1_at_30s,mean_20s_LP_Fp1-C3_at_30s,min_20s_LP_Fp1-C3_at_30s,max_20s_LP_Fp1-C3_at_30s,std_20s_LP_Fp1-C3_at_30s,max-min_20s_LP_Fp1-C3_at_30s,mean_20s_LP_C3-O1_at_30s,min_20s_LP_C3-O1_at_30s,max_20s_LP_C3-O1_at_30s,std_20s_LP_C3-O1_at_30s,max-min_20s_LP_C3-O1_at_30s,mean_20s_RP_Fp2-C4_at_30s,min_20s_RP_Fp2-C4_at_30s,max_20s_RP_Fp2-C4_at_30s,std_20s_RP_Fp2-C4_at_30s,max-min_20s_RP_Fp2-C4_at_30s,mean_20s_RP_C4-O2_at_30s,min_20s_RP_C4-O2_at_30s,max_20s_RP_C4-O2_at_30s,std_20s_RP_C4-O2_at_30s,max-min_20s_RP_C4-O2_at_30s,mean_20s_RR_Fp2-T4_at_30s,min_20s_RR_Fp2-T4_at_30s,max_20s_RR_Fp2-T4_at_30s,std_20s_RR_Fp2-T4_at_30s,max-min_20s_RR_Fp2-T4_at_30s,mean_20s_RR_T4-O2

In [15]:
FEATURES = df_train.drop(["seconds_eeg", "seconds_eeg_offset", "eeg_id"]).columns

In [16]:
FEATURES

['expert_consensus',
 'mean_10s_LL_Fp1-T3_at_20s',
 'min_10s_LL_Fp1-T3_at_20s',
 'max_10s_LL_Fp1-T3_at_20s',
 'std_10s_LL_Fp1-T3_at_20s',
 'max-min_10s_LL_Fp1-T3_at_20s',
 'mean_10s_LL_T3-O1_at_20s',
 'min_10s_LL_T3-O1_at_20s',
 'max_10s_LL_T3-O1_at_20s',
 'std_10s_LL_T3-O1_at_20s',
 'max-min__10s_LL_T3-O1_at_20s',
 'mean_10s_LP_Fp1-C3_at_20s',
 'min_10s_LP_Fp1-C3_at_20s',
 'max_10s_LP_Fp1-C3_at_20s',
 'std_10s_LP_Fp1-C3_at_20s',
 'max-min_10s_LP_Fp1-C3_at_20s',
 'mean_10s_LP_C3-O1_at_20s',
 'min_10s_LP_C3-O1_at_20s',
 'max_10s_LP_C3-O1_at_20s',
 'std_10s_LP_C3-O1_at_20s',
 'max-min_10s_LP_C3-O1_at_20s',
 'mean_10s_RP_Fp2-C4_at_20s',
 'min_10s_RP_Fp2-C4_at_20s',
 'max_10s_RP_Fp2-C4_at_20s',
 'std_10s_RP_Fp2-C4_at_20s',
 'max-min_10s_RP_Fp2-C4_at_20s',
 'mean_10s_RP_C4-O2_at_20s',
 'min_10s_RP_C4-O2_at_20s',
 'max_10s_RP_C4-O2_at_20s',
 'std_10s_RP_C4-O2_at_20s',
 'max-min_10s_RP_C4-O2_at_20s',
 'mean_10s_RR_Fp2-T4_at_20s',
 'min_10s_RR_Fp2-T4_at_20s',
 'max_10s_RR_Fp2-T4_at_20s',
 'std